# MLOps

parquet file ->(raw data) -> Data Preparation ->(features),(targets) -> Model Training -> (artifact) -> ML model

# Feature Store

hopswork.ai

generate API key to use the Python SDK to communicate with the feature store

In [1]:
from src.config import PROJECT_NAME, API_KEY


In [8]:
import hopsworks

project = hopsworks.login(project=PROJECT_NAME, api_key_value=API_KEY)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1152153


In [9]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [4]:
from src.paths import TRANSFORMED_DATA_DIR
import pandas as pd
data = pd.read_parquet(TRANSFORMED_DATA_DIR / 'yellow_tripdata_features_target.parquet')

In [6]:
data

,rides_previous_672,rides_previous_671,rides_previous_670,rides_previous_669,rides_previous_668,rides_previous_667,rides_previous_666,rides_previous_665,rides_previous_664,rides_previous_663,...,rides_previous_7,rides_previous_6,rides_previous_5,rides_previous_4,rides_previous_3,rides_previous_2,rides_previous_1,pickup_hour,PULocationID,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29 00:00:00,4,15.0
1,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,5.0,5.0,4.0,10.0,7.0,5.0,9.0,2022-01-29 23:00:00,4,9.0
2,0.0,3.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,...,13.0,6.0,8.0,7.0,8.0,5.0,5.0,2022-01-30 22:00:00,4,0.0
3,7.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,3.0,2.0,...,2.0,3.0,6.0,3.0,16.0,7.0,1.0,2022-01-31 21:00:00,4,1.0
4,3.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,2.0,5.0,3.0,8.0,3.0,2022-02-01 20:00:00,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-28 01:00:00,110,0.0
259840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-29 00:00:00,110,0.0
259841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-29 23:00:00,110,0.0
259842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-30 22:00:00,110,0.0
